### Projeto 1 - Ciência dos Dados

Nome: __João Lucas Brasileiro de Sousa Lopes__

Nome: __Láisa Camilly de Oliveira__

Nome: __Nathan Braga__

___
Carregando algumas bibliotecas:

In [84]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import unidecode

In [85]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\laisa\OneDrive\Área de Trabalho\Insper - Eng. Comp\2º semestre\C.Dados\Projeto 1\Projeto-1


Carregando a base de dados com as mensagens dos seus arquivos:

___
#### Classificador automático.
- O objetivo deste projeto é desenvolver um classificador baseado no algoritmo Naive Bayes, capaz de determinar se uma determinada frase pertence à área de Química ou Física.
- O dataset utilizado consiste em frases de ambas as disciplinas, associadas a rótulos que identificam a qual matéria elas pertencem.

___
#### Montando um Classificador Naive-Bayes.
<p>Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.</p>

- Criação de uma função responsável pela limpeza do dataset analisado, incluindo stopwords, acentos, e caracteres.

In [92]:
# funçao para limpeza de caracteres
import unidecode

def limpeza(texto):
    
    """
    Limpa o texto fornecido, tirando caracteres indesejados.
    """

    caracteres = '[´"!-.:?;\$\n'']'
    limpar = re.compile(caracteres)
    texto_limpo = re.sub(limpar, '', texto)

    texto_limpo = unidecode.unidecode(texto_limpo)

    stopwords = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]
    texto_limpo = texto_limpo.split()
    no_stop = ''
    for word in texto_limpo:
        if word not in stopwords:
            no_stop += word + " "
    
    return no_stop
    

In [93]:
# Criando nossa base de dados para treinamento e teste:
train = pd.read_csv('dados_treino_TRIO_joaolbsl.csv')

test = pd.read_csv('dados_teste_TRIO_joaolbsl.csv')

In [94]:
# Chamando a função criada para tratar a base de dados Train:
for i in range(0,train.shape[0]):
    train['Comment'][i] = limpeza(train['Comment'][i])
    train['Comment'][i] = train['Comment'][i].lower().strip()

In [95]:
# Separando o df para o topico Física e Química.
train_fis = train.loc[train['Topic'] == 'Physics', 'Comment'] 
train_qui = train.loc[train['Topic'] == 'Chemistry', 'Comment'] 

In [96]:
# Lista com todas as palavras que aparecem no df de física.
palavras_fis = []

for frase in train_fis:
    frase = frase.replace('\\n', ' ')
    frase = frase.split()

    for word in frase:
        palavras_fis.append(word)

# Lista com todas as palavras que aparecem no df de química.
palavras_qui = []

for frase in train_qui:
    frase = frase.replace('\\n', ' ')
    frase = frase.split()

    for word in frase:
        palavras_qui.append(word)


In [97]:
# Lista com palavras de física e química juntas.
todas_palavras = (palavras_fis + palavras_qui)

- Criação da função que calcula a probabilidade de tal palavra ser de tal tópico.

In [98]:
def calcula_prob (frase, palavras_topico, total_palavras_topico, todas_palavras):

    """
    Calcula a probabilidadea de uma palavra do topico de física ou química estar na frase.
    """

    prob = 1
    frase = frase.split()
    for palavra in frase:
        if palavra in todas_palavras:
            prob = prob * (palavras_topico.count(palavra) + 1) / (total_palavras_topico + len(todas_palavras))

    return prob

In [99]:
# Quantidade de palavras em Física e Química, para chamar na minha função.
total_palavras_fis = len(palavras_fis)
total_palavras_qui= len(palavras_qui)

In [100]:
# # Testando a função criada baseado no classificador, dado uma frase específica:
frase = "the physics of the universe"
prob_fis = calcula_prob(frase, palavras_fis, total_palavras_fis, todas_palavras)
prob_qui = calcula_prob(frase, palavras_qui, total_palavras_qui, todas_palavras)

# Vendo o valor:
print(f"Probabilidade de ser Física: {prob_fis}")
print(f"Probabilidade de ser Química: {prob_qui}")

Probabilidade de ser Física: 1.642953794776832e-15
Probabilidade de ser Química: 5.879988543545382e-18


In [101]:
# Todas as frases do df Treino:
frases_train = train['Comment']

# Todas as frases do df Teste:
frases_test = test['Comment']

___
#### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Treinamento.

In [102]:
# Fazendo com que nosso robô "Boot", calcule as probabilidades do df Train.
topico_esperado = []

for frase in frases_train:

    prob_fis = calcula_prob(frase, palavras_fis, total_palavras_fis, todas_palavras)
    prob_qui = calcula_prob(frase, palavras_qui, total_palavras_qui, todas_palavras)

    if prob_fis > prob_qui:
        topico_esperado.append("Physics")
    else:
        topico_esperado.append("Chemistry")

train['Boot'] = topico_esperado

tabela_frequencia_relativa_train = pd.crosstab(train['Topic'], train['Boot'], normalize = True, margins =True)

print(tabela_frequencia_relativa_train)

Boot       Chemistry   Physics       All
Topic                                   
Chemistry   0.521143  0.034571  0.555714
Physics     0.051143  0.393143  0.444286
All         0.572286  0.427714  1.000000


In [103]:
# Verdadeiros Positivos.
vp_train = tabela_frequencia_relativa_train['Chemistry']['Chemistry'] / tabela_frequencia_relativa_train['All']['Chemistry']
print(f'Vp:{vp_train}')

Vp:0.9377892030848328


In [104]:
# Falsos Positivos.
fp_train = tabela_frequencia_relativa_train['Chemistry']['Physics']/tabela_frequencia_relativa_train['All']['Physics']
print(f'Fp:{fp_train}')

Fp:0.11511254019292605


In [105]:
# Verdadeiros Negativos.
vn_train = 1 - fp_train
print(f'Vn: {vn_train}')

Vn: 0.884887459807074


In [106]:
# Acurácia de Treino.
acuracia_train = (tabela_frequencia_relativa_train['Chemistry']['Chemistry'] + tabela_frequencia_relativa_train['Physics']['Physics'])*100
print(f'Acurácia df.Train: {acuracia_train:.2f}')

Acurácia df.Train: 91.43


___
#### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Teste.

In [107]:
# Fazendo com que nosso robô "Boot", calcule as probabilidades do df Test.
topico_esperado = []

for frase in frases_test:

    prob_fis = calcula_prob(frase, palavras_fis, total_palavras_fis, todas_palavras)
    prob_qui = calcula_prob(frase, palavras_qui, total_palavras_qui, todas_palavras)

    if prob_fis > prob_qui:
        topico_esperado.append("Physics")
    else:
        topico_esperado.append("Chemistry")

test['Boot'] = topico_esperado

tabela_frequencia_relativa_test = pd.crosstab(test['Topic'], test['Boot'], normalize = True, margins = True)

print(tabela_frequencia_relativa_test)

Boot       Chemistry  Physics       All
Topic                                  
Chemistry   0.472667    0.102  0.574667
Physics     0.169333    0.256  0.425333
All         0.642000    0.358  1.000000


In [108]:
# Verdadeiros Positivos.
vp_test = tabela_frequencia_relativa_test['Chemistry']['Chemistry'] / tabela_frequencia_relativa_test['All']['Chemistry']
print(f'Vp: {vp_test}')

Vp: 0.8225058004640372


In [109]:
# Falsos Positivos. 
fp_test = tabela_frequencia_relativa_test['Chemistry']['Physics']/tabela_frequencia_relativa_test['All']['Physics']
print(f'Fp: {fp_test}')

Fp: 0.3981191222570533


In [110]:
# Verdadeiros Negativos.
vn_test = 1 - fp_test
print(f'Vn: {vn_test}')

Vn: 0.6018808777429467


In [111]:
# Acurácia do df.Test.
acuracia_test = (tabela_frequencia_relativa_test['Chemistry']['Chemistry'] + tabela_frequencia_relativa_test['Physics']['Physics'])*100
print(f'Acurácia df.Test: {acuracia_test:.2f}')

Acurácia df.Test: 72.87


___
- Verificando o comportamento do nosso robô da base de treino e da base de teste, e calculando a diferenç percentual entre os dois.

In [112]:
# Diferença de acurácias:
diferenca  = acuracia_train - acuracia_test
print(f'Diferenças do percentual de acurácia entre Train e Test: {diferenca:.2f}')

Diferenças do percentual de acurácia entre Train e Test: 18.56


___
#### Conclusão.

- Considerando a base de Testes, a qual foi analisada, é possível dizer que o classificador é preciso. Tendo em vista que o classificador obteve 73.87% de acurácia.
- 
- Para verificar a performance do classificador foram calculadas as acurácias utilizando as duas bases de dados, a base de treino e a base de testes, sendo a primeira aproximadamente 2.3 vezes maior que a segunda.
- A acurácia utilizando a base de treino foi de 86.9% e com a base de teste foi de 74.13%, sendo a diferença entre as duas igual a 12.83%.
- Para o aperfeiçoamento do classificador, foi necessário a implementação da limpeza das duas bases de dados, removendo os caracteres indesejados e fazendo com que assim as bases de dados estejam mais limpas.

___
#### Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
#### Aperfeiçoamento:

Leia atentamente a rubrica colocada no enunciado do Projeto 1 (última página). <br>
Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens, mas tendem a melhorar na classificação das mensagens. Ex: stemming, lemmatization, stopwords.
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar novas mensagens classificadas pelo próprio classificador como amostra de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* REFLETE criticamente sobre os resultados obtidos, identificando limitações do modelo e sugerindo possíveis melhorias ou diferentes abordagens com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa).
* DOCUMENTOU bem o código, com explicações claras para cada etapa do processo, incluindo o raciocínio por trás das decisões de modelagem e das transformações de dados.


___
#### Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[Natural Language Processing (Part 17)-Laplacian Smoothing](https://medium.com/@Coursesteach/natural-language-processing-part-17-laplacian-smoothing-7d4be71d0ded) **Mais simples**